# Creating a rotating globe to show instagram coverage

Following this [blog](https://makersportal.com/blog/2018/8/16/rotating-globe-in-python-using-basemap-toolkit)

In [ ]:
output_dir = '../../images/instagram-coverage-globe/50-places/'
output_file = 'globe-coverage.mp4'

png_dir = output_dir + 'png_dir_background/'
data_file = '../../data/wikivoyage/enriched/wikivoyage_destinations.csv'

background_color = '#003859'

In [ ]:
import datetime,matplotlib,time
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import shutil
import imageio

## Step 0: get the places that we featured

In [ ]:
# hard copy from excel instagram post list
featured_places = [734084, 487177, 695433, 212363, 834699, 544653, 311565, 430102, 407578, 482586, 968478, 546594, 892966, 595111, 440744, 999079, 916650, 644651, 108206, 539442, 507446, 731702, 179384, 627255, 698560, 832450, 334786, 258755, 981575, 560330, 993099, 209357, 464720, 522962, 689492, 767322, 733660, 228316, 169182, 220380, 171868, 700129, 159458, 473827, 145126, 749803, 298478, 184570, 762363, 539699, 487177, 850394, 983183, 900939, 380647, 840626, 110451, 766851, 222076]

In [ ]:
df = pd.read_csv(data_file).loc[lambda df: df['id'].isin(featured_places)]#[['lat', 'lng']]
points = [(v['lat'], v['lng']) for (k, v) in df.iterrows()]

## Step 1: Create function for plotting 3D globe

Note: for some reason a .mp4 made from images created with a dpi of 300 cannot be imported into iMovie. Set dpi to 200 and it will work...

For more info about how to set an exact pixel size image with matplotlib, see this [explanation](https://stackoverflow.com/questions/13714454/specifying-and-saving-a-figure-with-exact-size-in-pixels).

The scale argument in `bluemarble()` indicates the quality of the map. See [docs](https://matplotlib.org/basemap/api/basemap_api.html#mpl_toolkits.basemap.Basemap.bluemarble).

To add a glow effect to your scatter. Redraw the scatter a couple of times with different alpha. See [blog](https://towardsdatascience.com/cyberpunk-style-with-matplotlib-f47404c9d4c5).

In [ ]:
def create_image(lat, lon, points, filename, show_plot=False, background_color=None, dpi=200):
    
    fig = plt.figure(figsize=(16,9))
    if background_color:
        fig.patch.set_facecolor(background_color)

    # define color maps for water and land
    ocean_map = (plt.get_cmap('ocean'))(210)
    cmap = plt.get_cmap('gist_earth')

    # call the basemap and use orthographic projection at viewing angle
    m = Basemap(projection='ortho', lat_0=lat, lon_0=lon)    

    # coastlines, map boundary, fill continents/water, fill ocean, draw countries
    m.bluemarble(scale=1)   # to low skill yields ugly edges
#     m.drawcoastlines(color='white', linewidth=0.2)
#     m.drawmapboundary(fill_color=ocean_map)
#     m.fillcontinents(color=cmap(200),lake_color=ocean_map)
#     m.drawcountries()

    # scatter for plotting the lat/lon points
    for i in points:
        lat = i[0]
        lon = i[1]
        x,y = m(lon,lat)
        m.scatter(x, y, marker='o', color='#F16C21', s=25, alpha = 1)

        # For neon effect, redraw the data with low alpha and slighty increased linewidth:
        n_shades = 10
        diff_linewidth = 1.05
        alpha_value = 0.5 / n_shades
        for n in range(1, n_shades+1):
            m.scatter(x, y, marker='o', color='#F16C21', s=70,
                      linewidth=2+(diff_linewidth*n), alpha=alpha_value)
    
    # add margin if you want to place text below the image
#     plt.subplots_adjust(bottom=0.2, top=0.95)

    if background_color:
        plt.savefig(filename, facecolor=background_color, edgecolor=background_color, dpi=dpi)
    else:
        plt.savefig(filename, transparent=True, dpi=dpi)
    
    if show_plot:
        plt.show()
        
    plt.close()

In [ ]:
# set perspective angle
lat_viewing_angle = 10
lon_viewing_angle = 9

# create example image
# create_image(lat_viewing_angle, lon_viewing_angle, 
#              points, 
#              output_dir + 'orthographic_map_example_python.png', True, background_color)

## Step 2: Rotate earth and save pngs

In [ ]:
rotation_steps = 200

# create vector of longitutes to loop over
lon_viewing_angle = [-180, 180]
lon_vec = np.linspace(lon_viewing_angle[0],lon_viewing_angle[1],rotation_steps)
start_index = min(range(len(lon_vec)), key=lambda i: abs(lon_vec[i]+lat_viewing_angle))
lon_steps = np.hstack([lon_vec[start_index:],lon_vec[:start_index]])

In [ ]:
# remove old images before placing new
if os.path.exists(png_dir):
    shutil.rmtree(png_dir)
os.makedirs(png_dir)

# loop through the longitude vector above
gif_indx = 0
for lon in lon_steps:    
    create_image(20, lon, points, png_dir + 'frame_'+str(gif_indx)+'_.png', background_color=background_color)
    gif_indx+=1

## Step 3: Make Gif/Movie

`gif_maker` function below taken from this [github](https://github.com/makerportal/gifly) and adapted a bit.

The default '.mp4' method creates videos in an AVC format. However, those are [not compatible with iMovie](https://discussions.apple.com/thread/250754287). So use the recommended settings as described [here](https://stackoverflow.com/questions/52948735/quality-loss-in-imageio).

If all that doesn't work, just [try another video library](https://stackoverflow.com/questions/53214221/how-to-create-a-video-from-images/53214695)

In [ ]:
def gif_maker(gif_name, png_dir):
    # define some GIF parameters
    frame_length = 0.5 # seconds between frames
    end_pause = 4 # seconds to stay on last frame
    # define MP4 parameters
    fps = 20

    # list the images for in the animation
    images,image_file_names = [],[]
    for file_name in os.listdir(png_dir):
        if file_name.endswith('.png'):
            image_file_names.append(file_name)       
    sorted_files = sorted(image_file_names, key=lambda y: int(y.split('_')[1]))
    
    # loop through files, join them to image array, and write to GIF called 'wind_turbine_dist.gif'
    for ii in range(0,len(sorted_files)):       
        file_path = os.path.join(png_dir, sorted_files[ii])
        if ii==len(sorted_files)-1:
            for jj in range(0,int(end_pause/frame_length)):
                images.append(imageio.imread(file_path))
        else:
            images.append(imageio.imread(file_path))

    if gif_name.split('.')[-1] == 'gif':
        # the duration is the time spent on each image (1/duration is frame rate)
        imageio.mimsave(gif_name, images,'GIF',duration=frame_length)
    else:  # make mp4
        writer = imageio.get_writer(gif_name, fps=fps)
        for im in [png_dir + file for file in sorted_files]:
            writer.append_data(imageio.imread(im))
        writer.close()


In [ ]:
# gif_maker(output_dir + 'test.gif', output_dir + 'png_dir/')
gif_maker(output_dir + output_file, png_dir)

Done. The rest of the video editing is done in iMovie.